# lexsub: default program

In [1]:
from default import *
import os

## Run the default solution on dev

In [2]:
lexsub = LexSub(os.path.join('data','glove.6B.100d.magnitude'))
output = []
with open(os.path.join('data','input','dev.txt')) as f:
    for line in f:
        fields = line.strip().split('\t')
        output.append(" ".join(lexsub.substitutes(int(fields[0].strip()), fields[1].strip().split())))
print("\n".join(output[:10]))

sides edge bottom front club line both back place corner
sides edge bottom front club line both back place corner
sides edge bottom front club line both back place corner
sides edge bottom front club line both back place corner
sides edge bottom front club line both back place corner
sides edge bottom front club line both back place corner
sides edge bottom front club line both back place corner
sides edge bottom front club line both back place corner
sides edge bottom front club line both back place corner
sides edge bottom front club line both back place corner


## Evaluate the default output

In [3]:
from lexsub_check import precision
with open(os.path.join('data','reference','dev.out'), 'rt') as refh:
    ref_data = [str(x).strip() for x in refh.read().splitlines()]
print("Score={:.2f}".format(100*precision(ref_data, output)))

Score=27.89


## Documentation

Write some beautiful documentation of your program here.

## Retrofitting Word Vector

In [9]:
import os
# read words from lexicon
def read_word_vector(filename):
    wordVec={}
    for line in open(filename, 'r'):
        wordC = line.strip().lower().split()
        temp = [w for w in wordC[1:]]
        if wordC[0] not in wordVec:
            wordVec[wordC[0]] = temp
        else:
            wordVec[wordC[0]] += temp
    return wordVec
path = os.path.dirname(os.getcwd())
pathname = os.path.join(path,'data/lexicons','ppdb-xl.txt')             
lexicon = read_word_vector(pathname)

In [10]:
def retrofit(wordVecs, lexicon, T=10, alpha=1, beta=1):
    newWordVecs = {}
    wvVocab = set()
    for word, it in wordVecs:
        wvVocab.add(word)
        newWordVecs[word] = it
    for i in range(T):
        for j, word in enumerate(wvVocab):
            temp = np.zeros(newWordVecs[word].shape)
            if word in lexicon:
                count = 0
                LoopVocab = lexicon[word]
                for w in LoopVocab:
                    if w in newWordVecs:
                        temp += beta * newWordVecs[w]
                        count += 1
                newWordVecs[word] = ((temp + (alpha * wordVecs.query(word)))) / (count + alpha)
    return newWordVecs



In [14]:
import pymagnitude
from pymagnitude import converter
import numpy as np
from gensim import scripts
import optparse, tqdm
from gensim.scripts.glove2word2vec import glove2word2vec

def convert_to_magnitude_and_retrofit(mag_path):
    
#     path = os.path.dirname(os.getcwd())
#     print(mag_path)
    path = os.getcwd()
    if os.getcwd().split('/')[-1] == 'answer':
        path = os.path.dirname(os.getcwd())
    LL_path = os.path.join(path,'data/lexicons','ppdb-xl.txt')
#     print("word vector reading.\n")
    lexicon = read_word_vector(LL_path)
    
#     print("word vector read.\n")
    wv = pymagnitude.Magnitude(mag_path)
#     print("retrofitting.\n")
    lexicon_retrofitted = retrofit(wv, lexicon)
    
#     print("retrofitted.\n")

    retrofitpath = os.path.join(path,'data','glove.6B.100d.retrofit.txt')
    if os.path.exists(os.path.join(path,'data','glove.6B.100d.retrofit.magnitude')):
        wvecs=pymagnitude.Magnitude(os.path.join(path,'data','glove.6B.100d.retrofit.magnitude'))
#         print("retrofit.magnitude exists\n")
    else:
#         print("retrofit.magnitude does/ not exist\n")
        with open(retrofitpath, 'w') as f:
            for word, embedding in lexicon_retrofitted.items():
                s = word
                for num in embedding:
                    s += " " + str(num)
                s += '\n'
                f.write(s)
            target_file=os.path.join(path,'data','glove.6B.100d.retrofit.magnitude')

            # target_file=os.path.join(path,'data',mag_path)
            converter.convert(retrofitpath, target_file)
#         print(os.path.join(path,'data','glove.6B.100d.retrofit.magnitude'))
        wvecs = pymagnitude.Magnitude(os.path.join(path,'data','glove.6B.100d.retrofit.magnitude'))
#         print("retrofit.magnitude exists\n")
#     wvecs = pymagnitude.Magnitude(os.path.join(path,mag_path))
    return wvecs
mag_path = os.path.join(os.path.dirname(os.getcwd()), 'data','glove.6B.100d.retrofit.magnitude')
convert_to_magnitude_and_retrofit(mag_path)

In [15]:
from lexsub_check import precision
from lexsub import *
import os
lexSub=LexSub(os.path.join(os.path.dirname(os.getcwd()), 'data','glove.6B.100d.retrofit.magnitude'))
output = []
with open(os.path.join('../data','input','dev.txt')) as f:
    for line in f:
        fields = line.strip().split('\t')
        output.append(" ".join(lexSub.substitutes(int(fields[0].strip()), fields[1].strip().split())))
print("\n".join(output[:10]))

from lexsub_check import precision
with open(os.path.join('../data','reference','dev.out'), 'rt') as refh:
    ref_data = [str(x).strip() for x in refh.read().splitlines()]
print("Score={:.2f}".format(100*precision(ref_data, output)))

sides edge bottom part hand place close tip under below
sides edge bottom part hand place close tip under below
sides edge bottom part hand place close tip under below
sides edge bottom part hand place close tip under below
sides edge bottom part hand place close tip under below
sides edge bottom part hand place close tip under below
sides edge bottom part hand place close tip under below
sides edge bottom part hand place close tip under below
sides edge bottom part hand place close tip under below
sides edge bottom part hand place close tip under below
Score=52.85


## Analysis

Do some analysis of the results. What ideas did you try? What worked and what did not?